In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 7.3 MB/s 
     |████████████████████████████████| 7.6 MB 54.9 MB/s 
     |████████████████████████████████| 182 kB 72.4 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoTokenizer, AutoModel
from scipy.special import softmax
import _pickle as cPickle
import numpy as np
import torch

In [4]:
#from https://huggingface.co/sberbank-ai/sbert_large_mt_nlu_ru
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [5]:
device = torch.device('cuda')

__Загрузка модели ArgBERT__

In [6]:
# Загрузка токенизатора
model_name = "sberbank-ai/sbert_large_mt_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/331 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
# Путь к модели ArgBERT
path = "/content/drive/MyDrive/"
model_path = path + "ArgBERT"

In [ ]:
# Загрузка модели и ее перенос на GPU
model = AutoModel.from_pretrained(model_path).to(device)

__Загрузка классификаторов Random Forest__

In [9]:
RF_models = path + "RF_models/"
aspects = [
    "Безопасность",
    "Влияние на здоровье",
    "Влияние на психику",
    "Надежность",
    "Отношение властей",
    "Уровень жизни",
    "Эффективность"
]
models = []
for aspect in aspects:
    with open(RF_models+aspect+'.pkl', 'rb') as f:
        models.append(cPickle.load(f))
models

[RandomForestClassifier(n_estimators=500, random_state=42),
 RandomForestClassifier(class_weight='balanced', n_estimators=500,
                        random_state=42),
 RandomForestClassifier(class_weight='balanced', n_estimators=500,
                        random_state=42),
 RandomForestClassifier(class_weight='balanced', n_estimators=500,
                        random_state=42),
 RandomForestClassifier(class_weight='balanced', n_estimators=500,
                        random_state=42),
 RandomForestClassifier(class_weight='balanced', max_depth=5, n_estimators=500,
                        random_state=42),
 RandomForestClassifier(class_weight='balanced', max_depth=5, n_estimators=500,
                        random_state=42)]

__Аргументы для классификации__

In [10]:
arguments = ["Польза перерывов для работы очевидна – отдых восстанавливает силы и повышает продуктивность.",
             "Оставлять приборы в розетке под напряжением опасно — может начаться утечка тока и неконтролируемый разогрев.",
             "Как уже было сказано, засыпание с телевизором подвергает вас воздействию синего света от электронных устройств, что ухудшает качество сна, подавляя производство мелатонина и откладывает начало сна."]

__Получение векторного представления предложений__

In [21]:
%%time
vectors_arguments = []
for argument in arguments:
    encoded_input = tokenizer(argument, padding=True, truncation=True, max_length=200, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input.to("cuda"))
    vector_argument = mean_pooling(model_output, encoded_input['attention_mask']) 
    vectors_arguments.append(vector_argument)

CPU times: user 64.8 ms, sys: 0 ns, total: 64.8 ms
Wall time: 65.8 ms


__Классификация аргументов__

In [23]:
%%time
for argument, vector_argument in zip(arguments, vectors_arguments):
    predicts = []
    for rf_model in models:
        predicts.append(rf_model.predict_proba(vector_argument.cpu())[0][1])
    predicts = softmax(predicts)

    print(f"Аргумент: {argument}")
    for aspect, pred in zip(aspects, predicts):
        print(f"{aspect}: {round(pred, 4)}")
    ind_aspect = np.argmax(predicts)
    predict_aspect = aspects[ind_aspect]
    print(f"\nПредсказанный аспект: {predict_aspect}")
    print("\n", "*"*100, "\n")

Аргумент: Польза перерывов для работы очевидна – отдых восстанавливает силы и повышает продуктивность.
Безопасность: 0.1429
Влияние на здоровье: 0.1461
Влияние на психику: 0.1357
Надежность: 0.134
Отношение властей: 0.1322
Уровень жизни: 0.1358
Эффективность: 0.1733

Предсказанный аспект: Эффективность

 **************************************************************************************************** 

Аргумент: Оставлять приборы в розетке под напряжением опасно — может начаться утечка тока и неконтролируемый разогрев.
Безопасность: 0.1601
Влияние на здоровье: 0.1572
Влияние на психику: 0.1402
Надежность: 0.1411
Отношение властей: 0.1271
Уровень жизни: 0.1261
Эффективность: 0.1481

Предсказанный аспект: Безопасность

 **************************************************************************************************** 

Аргумент: Как уже было сказано, засыпание с телевизором подвергает вас воздействию синего света от электронных устройств, что ухудшает качество сна, подавляя производ